# Vorbereitung der Daten für maschinelles Lernen
Es ist an der Zeit, die Daten für Algorithmen des maschinellen Lernens vorzubereiten. Aus mehreren guten Gründen sollten Sie für die Datenvorbereitung Funktionen schreiben:
- Dadurch können Sie die Transformationen leicht auf jedem beliebigen Datensatz reproduzieren (z.B, wenn Sie das nächste Mal einen neuen Datensatz erhalten)
- Sie werden nach und nach eine Bibliothek von Transformationsfunktionen aufbauen, die Sie wiederverwenden können in zukünftigen Projekten
- Sie können diese Funktionen in Ihrem Produktivsystem verwenden, um die neuen Daten zu transformieren, bevor Sie Ihren Algorithmus damit versorgen
- Auf diese Weise können Sie einfach verschiedene Transformationen ausprobieren und sehen, welche Kombination von Transformationen am besten funktioniert

Wir beginnen mit einem frischen Datensatz:

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

# Datenbeschaffung / Einlesen einer csv Datei wie in Abschnitt 2 beschrieben
def load_housing_data():
    csv_path = os.path.join("datasets/housing/housing.csv")
    return pd.read_csv(csv_path)


housing = load_housing_data()
# Erstellung income category Attribut mit fünf Kategorien
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

# Basierend auf dem Kategorie-Attribut wird nun eine stratifizierte Stichprobe gezogen
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    
strat_train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16512 entries, 12655 to 19773
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16354 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   median_house_value  16512 non-null  float64
 9   ocean_proximity     16512 non-null  object 
 10  income_cat          16512 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.5+ MB


Als erstes erstellen wir zwei Datenobjekte für das Trainieren des Algorithmus. Ein Datenobjekt enthält alle determinierenden Attribute (housing), exklusive des Lables bzw. des Zielattributes. Das zweite Datenobjekt soll ausschließlich die Zielattribute, in unserem Fall also die Hauswerte, enthalten. 

In [4]:
housing = strat_train_set.drop("median_house_value",axis=1)

housing_labels = strat_train_set["median_house_value"].copy()


# Data Cleaning
Die meisten Algorithmen des maschinellen Lernens können nicht mit fehlenden Werten arbeiten, also lassen Sie uns ein paar Funktionen erstellen, um dieses Problem zu beheben. <br>
Aufgabe 1: Lassen Sie sich nochmal eine Übersicht über die housing-Datensätze anzeigen. <br>
Aufgabe 2: Wie erkennen Sie, ob ein Attribut fehlende Werte besitzt? Welches Attribut hat fehlende Werte?

Sie haben drei Möglichkeiten mit fehlenden Werten umzugehen:
- Sie können die entsprechenden Bezirke entfernen (Option 1)
- Sie können das gesamte Attribut entfernen (Option 2)
- Sie können die Werte manuell belegen (Null, den Mittelwert, den Median usw.) (Option 3)

Dies können Sie leicht mit den Methoden **dropna(), drop() und fillna() von DataFrame** erreichen:

In [13]:
# Option 1
#housing.dropna(subset=["total_bedrooms"]) 
# Option 2
#housing.drop("total_bedrooms", axis=1) 
# Option 3
median = housing['total_bedrooms'].median()
housing['total_bedrooms'].fillna(median, inplace=True)
housing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16512 entries, 12655 to 19773
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
 9   income_cat          16512 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1.4+ MB


Eine weitere Möglichkeit, fehlende Werte zu ersetzen stellt die Imputer-Methode bereit. Details erhalten Sie hier: https://scikit-learn.org/0.16/modules/generated/sklearn.preprocessing.Imputer.html.
        

# Behandlung von Text und Kategorie-Attributen
Eben haben wir das kategorische Attribut Ozean_Nähe ausgelassen, weil es ein Textattribut ist, so dass wir seinen Median nicht berechnen können. Die meisten Algorithmen des maschinellen Lernens arbeiten jedoch lieber mit Zahlen. Also lassen Sie uns Textattribute in Zahlen umwandeln. Scikit-Learn stellt für diese Aufgabe einen Transformator namens **LabelEncoder** zur Verfügung.

In [14]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

housing_cat = housing["ocean_proximity"]
housing_cat_encoded = encoder.fit_transform(housing_cat)
housing_cat_encoded

array([1, 4, 1, ..., 0, 0, 1])

Das ist besser: Jetzt können wir diese numerischen Daten in jedem ML-Algorithmus verwenden. Aber was bedeuten die Zahlen? <br>Aufgabe 3: Wie bekommen Sie die Zuordnungen der Bezeichnungen zu den Zahlen wiedergegeben? (Hinweis: Der Encoder arbeitet mit dem Attribut classes_)

In [15]:
# Hier sollte Ihr Code rein, um die Bezeichnungszuordnungen zu bekommen
encoder.classes_


array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype=object)

Das Problem mit nummerischen Werten in maschinellem Lernen ist, dass eine gewisse Nähe der Werte in Abhängigkeit zu ihrer zhalenmäßigen Distanz angenommen wird. In unserem Fall würde der Algorithmus also annehmen, dass zwischen den Werten 0 und 1 eine gewisse Ähnlichkeit besteht, was in unserem Beispiel falsch ist. Um diesen Effekt zu vermeiden, bietet Scikit-Learn den **OneHotEncoder** an. Die Methode erstellt ein binäres Attribut (nur 0 oder 1 Werte möglich) für jede Attributsausprägung. Also beispielsweise das Attribut "INLAND" mit dem Wert 0, wenn der Ortsbereich sich am Meer befindet.

In [16]:
from sklearn.preprocessing import OneHotEncoder

encoder2 = OneHotEncoder()

# Der Output von fit_transform ist ein Objekt vom Typ Sparse matrix mit binären Werten. 
housing_cat_1hot = encoder2.fit_transform(housing_cat_encoded.reshape(-1,1))

# Umwandlung einer dünn besetzten Matrix , in der nur die 1en gespeiochert werden in eine dicht 
# besetzte Matrix mit 0en un 1en
housing_cat_1hot = housing_cat_1hot.toarray()
housing_cat_1hot


array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

# Eigene Transformatoren bauen
Obwohl Scikit-Learn viele nützliche Transformatoren bietet, müssen Sie manchmal auch Ihre eigenen Transformatoren schreiben. Zum Beispiel für Aufgaben wie benutzerdefinierte Bereinigungsoperationen oder das Kombinieren bestimmter Attribute. Dafür müssen Sie eine Klasse erstellen und drei Methoden implementieren: 
- fit() (selbstrückkehrend),
- transform() und 
- fit_transform(). 

Hier ist zum Beispiel eine kleine Transformatorklasse, die Folgendes hinzufügt die kombinierten Attribute, die wir vorhin besprochen haben:

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self # nothing else to do

    # Da die Methode nur die Werte übergeben bekommt. muss auf das Array mit Spaltennummern (rooms_ix etc.) zugegriffen werden
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
# Das Objekt bekommt über housing.values nur die Werte übergeben. Die Attributsbezeichnungen nicht.
housing_extra_attribs = attr_adder.transform(housing.values)

housing_extra_attribs



array([[-121.46, 38.52, 29.0, ..., 2.0, 5.485835694050992,
        3.168555240793201],
       [-117.23, 33.09, 7.0, ..., 5.0, 6.927083333333333,
        2.6236979166666665],
       [-119.04, 35.37, 44.0, ..., 2.0, 5.3933333333333335,
        2.223333333333333],
       ...,
       [-122.72, 38.44, 48.0, ..., 3.0, 4.1104651162790695,
        2.6627906976744184],
       [-122.7, 38.31, 14.0, ..., 3.0, 6.297405189620759,
        2.411177644710579],
       [-122.14, 39.97, 27.0, ..., 3.0, 5.477157360406092,
        3.1725888324873095]], dtype=object)

In diesem Beispiel hat der Transformator einen Hyperparameter, *add_bedrooms_per_room*, der standardmäßig auf True gesetzt ist (es ist oft hilfreich, sinnvolle Vorgaben zu machen). Mit diesem Hyperparameter können Sie leicht herausfinden, ob das Hinzufügen dieses Attributs dem ML-Algorithmus hilft oder nicht. Allgemeiner gesagt, Sie können einen Hyperparameter hinzufügen, um jeden Datenvorbereitungsschritt zu steuern, bei dem Sie sich nicht 100%ig sicher sind. Je mehr Sie diese Datenvorbereitungsschritte automatisieren, desto mehr Kombinationen können Sie automatisch ausprobieren, wodurch es viel wahrscheinlicher wird, dass Sie eine treffende Kombination finden (und Sie viel Zeit sparen).

Aufgabe 4: Lassen Sie sich die Werte vom housing-Datensatz anzeigen und vergleichen Sie die Ausgabe mit den Werten in housing_extra_attribs. Was fällt Ihnen auf?

# Datenskallierung
Eine der wichtigsten Transformationen, die Sie auf Ihre Daten anwenden müssen, ist die Daten oder *Feature-Skalierung*. Von wenigen Ausnahmen abgesehen, schneiden die Algorithmen des maschinellen Lernens nicht gut ab, wenn die eingegebenen numerischen Attribute sehr unterschiedliche Skalen haben. Dies ist der Fall für die housing-Daten: Die Gesamtzahl der Räume reicht von etwa 6 bis 39.320, während der Median der Einkommen nur zwischen 0 und 15 liegt. Beachten Sie, dass die Skalierung der Zielwerte im Allgemeinen nicht erforderlich ist.
Es gibt zwei gängige Methoden, um alle Attribute auf die gleiche Skala zu bringen: **min-max Skalierung** und **Standardisierung**.

Die **Min-Max-Skalierung** (viele nennen dies Normalisierung) ist recht einfach: Die Werte sind verschoben und neu skaliert, so dass sie schließlich von 0 bis 1 reichen. Wir tun dies, indem wir den Min-Wert substrahieren und durch den Max-Wert minus den Min.-Wert dividieren. Scikit-Learn bietet eine Transformator dafür: den **MinMaxScaler**. Er hat einen Feature_Bereichs-Hyperparameter, mit dem Sie den Bereich ändern können, wenn Sie aus irgendeinem Grund nicht den Bereich von 0-1 haben wollen. 

Die **Standardisierung** ist ganz anders: Zuerst wird der Mittelwert substrahiert (standardisierte Werte haben immer einen Mittelwert von Null), und dann wird durch die Varianz geteilt, so dass die resultierende Verteilung eine Einheitsvarianz aufweist. Anders als bei der Min-Max-Skalierung werden bei der Standardisierung nicht Werte an einen bestimmten Bereich gebunden, was für einige Algorithmen ein Problem darstellen kann (z.B, neuronale Netze erwarten oft einen Eingabewert im Bereich von 0 bis 1). Allerdings ist die Standardisierung viel weniger von Ausreißern betroffen. Nehmen wir beispielsweise an, ein Distrikt hätte ein durchschnittliches Einkommen gleich 100 (versehentlich). Die Min-Max-Skalierung würde dann alle anderen vernichten. Werte von 0-15 bis hinunter zu 0-0,15, während die Standardisierung nicht wesentlich beeinträchtigt würde. Scikit-Learn stellt einen Transformator namens **StandardScaler** für die Normung zur Verfügung.

**Wichtig** 
Bitte beachten Sie, dass Skalierungen **ausschließlich auf den Trainingsdaten** angewendet werden, nicht auf den Testdaten oder den gesamten Grunddaten.

# Transformations Pipelines
Wie wir nun wissen, müssen zahlreiche Transformationen auf den Ursprungsdaten ausgeführt werden, damit sie in einem ML-Algorithmus genutzt werden können. Glücklicherweise bietet Scikit-Learn auch hierfür eine Klasse, die die Erstellung solcher Transformations Pipelines (data pipeline) unterstützt: sklearn.pipeline. Das folgende Beispiel verdeutlicht die Funktionsweise.

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Erstellen eines Dataframes ohne kategorielle Attribute
housing_num = housing.drop("ocean_proximity", axis=1)

# SimpleImputer ersetzt alle fehlenden Werte mit Ersatzwerten, in diesem Fall dem Median aller Werte der Spalte
# CombinedAttributesAdder ist die Transformationsklasse, die Attribute zusammensetzt (Bsp Anzahl Schlafzimmer / Anzahl Räume)
# StandardScaler skalliert alle Daten
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())])

housing_num_tr = num_pipeline.fit_transform(housing_num)

housing_num_tr


array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.01739526,
         0.00622264, -0.12112176],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.56925554,
        -0.04081077, -0.81086696],
       [ 0.26758118, -0.1259716 ,  1.22045984, ..., -0.01802432,
        -0.07537122, -0.33827252],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ..., -0.5092404 ,
        -0.03743619,  0.32286937],
       [-1.56080303,  1.2492109 , -1.1653327 , ...,  0.32814891,
        -0.05915604, -0.45702273],
       [-1.28105026,  2.02567448, -0.13148926, ...,  0.01407228,
         0.00657083, -0.12169672]])

**Aufgabe 5:** Lassen Sie sich die Inhalte von housing_num und von housing_num_tr ausgeben und vergleichen Sie den Output. Was fällt Ihnen auf?

In [22]:
print(housing_num_tr)

[[-0.94135046  1.34743822  0.02756357 ...  0.01739526  0.00622264
  -0.12112176]
 [ 1.17178212 -1.19243966 -1.72201763 ...  0.56925554 -0.04081077
  -0.81086696]
 [ 0.26758118 -0.1259716   1.22045984 ... -0.01802432 -0.07537122
  -0.33827252]
 ...
 [-1.5707942   1.31001828  1.53856552 ... -0.5092404  -0.03743619
   0.32286937]
 [-1.56080303  1.2492109  -1.1653327  ...  0.32814891 -0.05915604
  -0.45702273]
 [-1.28105026  2.02567448 -0.13148926 ...  0.01407228  0.00657083
  -0.12169672]]


Sie haben jetzt eine Pipeline für numerische Werte. Wir haben auch bereits einen Weg kennen gelernt, um kategorielle Attribute in binäre Attribute umzuwandeln. Eine weitere Möglichkeit bietet der *LabelBinarizer*, den wir gleich kennen lernen werden. 

Die zentrale Frage aber ist: *Wie kann man diese Transformationen zu einer einzigen Pipeline zusammenführen?* Scikit-Learn bietet hierfür eine **FeatureUnion-Klasse** an. Sie geben ihr eine Liste von Transformatoren (die auch ganze Transformator-Pipelines sein können). Wenn seine transform() Methode aufgerufen wird, führt sie die transform()-Methode jedes Transformators parallel aus, wartet auf ihre Ausgabe, verkettet sie dann und gibt das Ergebnis zurück Eine vollständige Pipeline-Behandlung sowohl numerischer als auch kategorialer Attribute könnte so aussehen:

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator , TransformerMixin

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

# Klasse für die Auswahl nummerischer und kategorieller Spalten
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

# Pipeline für ide Verarbeitung nummerischer Attribute
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
    ])

# Pipeline für die Verarbeitung kategorieller Attribute
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('label_binarizer', OneHotEncoder()),
    ])

# Zusammensetzen der Teil-Pipelines
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
    ])

# Bis hierher arbeitet die pipeline noch nicht mit echten Daten. Sie verfügt nur über das Wissen der Attribute und der
# Transformationsfunktionen. Erst jetzt werden der Pipeline echte housing-Daten übergeben:
housing_prepared = full_pipeline.fit_transform(housing)

